In [52]:
from buildingmotif import BuildingMOTIF
bm = BuildingMOTIF("sqlite://") 

In [117]:
from rdflib import Namespace, Literal
from buildingmotif.dataclasses import Model

# create the namespace
BLDG = Namespace('test:building/')

# Create the model! This will raise an exception if the namespace is not syntactically valid.
model = Model.create(BLDG, description="This is a test model for a simple building") 

In [54]:
print(model.graph.serialize())

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<test:building/> a owl:Ontology .




# Loading Libraries

In [55]:
# load a library
from buildingmotif.dataclasses import Library
brick = Library.load(ontology_graph="Brick-full.ttl")
from buildingmotif.namespaces import BRICK

    

2023-01-16 22:00:35,579 | root |  WARNING: Warning: could not find dependee http://data.ashrae.org/bacnet/2020#BACnetDevice
2023-01-16 22:00:35,579 | root |  WARNING: Warning: could not find dependee https://brickschema.org/schema/Brick/ref#ifcProject


# Exploring a Template

In [56]:
# import this to make writing URIs easier
"""
# get template
ahu_template = brick.get_template_by_name(BRICK.AHU)



print(ahu_template)
# print template parameters

print("The template has the following parameters:")
for param in ahu_template.parameters:
    print(f"  {param}")
"""

'\n# get template\nahu_template = brick.get_template_by_name(BRICK.AHU)\n\n\n\nprint(ahu_template)\n# print template parameters\n\nprint("The template has the following parameters:")\nfor param in ahu_template.parameters:\n    print(f"  {param}")\n'

In [57]:
#print(ahu_template.body.serialize())


# Evaluating a Template

In [58]:
"""
ahu_name = "AHU_AC"
ahu_binding_dict = {"name": BLDG[ahu_name]}
ahu_graph = ahu_template.evaluate(ahu_binding_dict)

# ahu_graph is just an instance of rdflib.Graph
print(ahu_graph.serialize())
"""

'\nahu_name = "AHU_AC"\nahu_binding_dict = {"name": BLDG[ahu_name]}\nahu_graph = ahu_template.evaluate(ahu_binding_dict)\n\n# ahu_graph is just an instance of rdflib.Graph\nprint(ahu_graph.serialize())\n'

# Adding Evaluated Templates to the Model

In [59]:
"""
model.add_graph(ahu_graph)
print(model.graph.serialize())
"""

'\nmodel.add_graph(ahu_graph)\nprint(model.graph.serialize())\n'

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [118]:
df=pd.read_csv('entity_relations_v4.csv')
filename  = '/Users/lazlopaul/Desktop/VolttronSemantics/files/entity_table.csv'
topics = pd.read_csv(filename)[['URI','base_topic']]
df = df.merge(topics, left_on = 'subjectURI', right_on = 'URI')

In [119]:
df['pointURI'] = np.zeros(len(df.index))
df['topic'] = np.zeros(len(df.index))

In [120]:
for i in df.index:
    if df.loc[i, 'relation'] == 'hasPoint':
        df.loc[i, 'topic'] = df.loc[i, 'base_topic'] + '/' + df.loc[i, 'objectURI']
        df.loc[i, 'objectURI'] = df.loc[i, 'subjectURI'] + '.' + df.loc[i, 'objectURI']
        # df.loc[i, 'pointURI'] = df.loc[i, 'subjectURI'] + '.' + df.loc[i, 'objectURI']


In [121]:
import re
def uni_encode(mystring):
    mystring=str(mystring)
    mystring=re.sub(r'[^\w\s]', '', mystring)
    mynewstring=mystring.replace(" ", "_")
    return mynewstring
#df=df.applymap(lambda x: uni_encode(x) if pd.notnull(x) else x)

In [122]:

#print(equipment_template)

In [123]:
length_dataframe=len(df)
#print(length_dataframe)

for row_df in range(0,length_dataframe):
    for column_df in [0,3]:
        brick_class=df.iloc[row_df,column_df+1]
        dummy_code = f"equipment_template= brick.get_template_by_name(BRICK.{brick_class})"
        exec(dummy_code)
        equipment_name =df.iloc[row_df,column_df]
        equipment_binding_dict = {"name": BLDG[equipment_name]}
        equipment_graph = equipment_template.evaluate(equipment_binding_dict)
        model.add_graph(equipment_graph)




In [124]:
for row_df in range(0,length_dataframe):
    equipment_name1 =df.iloc[row_df,0]
    equipment_name2 =df.iloc[row_df,3]
    BRICK_relation=df.iloc[row_df,2]
    dummy_code=f"model.graph.add((BLDG['{equipment_name1}'], BRICK.{BRICK_relation}, BLDG['{equipment_name2}']))"
    exec(dummy_code)
        
        

In [125]:
"""

# templates
oa_ra_damper_template = brick.get_template_by_name(BRICK.Damper)
fan_template = brick.get_template_by_name(BRICK.Supply_Fan)
clg_coil_template = brick.get_template_by_name(BRICK.Cooling_Coil)

# add fan
fan_name = f"{ahu_name}-Fan"
fan_binding_dict = {"name": BLDG[fan_name]}
fan_graph = fan_template.evaluate(fan_binding_dict)
model.add_graph(fan_graph)

# add outdoor air/return air damper
oa_ra_damper_name = f"{ahu_name}-Damper"
oa_ra_damper_binding_dict = {"name": BLDG[oa_ra_damper_name]}
oa_ra_damper_graph = oa_ra_damper_template.evaluate(oa_ra_damper_binding_dict)
model.add_graph(oa_ra_damper_graph)

# add clg coil
clg_coil_name = f"{ahu_name}-Clg_Coil"
clg_coil_binding_dict = {"name": BLDG[clg_coil_name]}
clg_coil_graph = clg_coil_template.evaluate(clg_coil_binding_dict)
model.add_graph(clg_coil_graph)

# connect fan, damper, and clg coil to AHU
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[oa_ra_damper_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[fan_name]))
model.graph.add((BLDG[ahu_name], BRICK.hasPart, BLDG[clg_coil_name]))
"""
# print model to confirm components were added and connected
print(model.graph.serialize())


@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<test:building/> a owl:Ontology .

<test:building/Bldg> a brick:Building ;
    brick:hasPart <test:building/AdminSpace>,
        <test:building/Office> .

<test:building/Boiler2> a brick:Boiler ;
    brick:feeds <test:building/HeatEx2> .

<test:building/FCU1> a brick:FCU ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPoint <test:building/FCU1.fanLevel>,
        <test:building/FCU1.mode>,
        <test:building/FCU1.on>,
        <test:building/FCU1.targetTemperature>,
        <test:building/FCU1.temperature>,
        <test:building/FCU1.timeOfTemperature> .

<test:building/FCU2> a brick:FCU ;
    brick:feeds <test:building/Zone1> ;
    brick:hasPoint <test:building/FCU2.fanLevel>,
        <test:building/FCU2.mode>,
        <test:building/FCU2.on>,
        <test:building/FCU2.targetTemperature>,
        <test:building/FCU2.temperature>,
        <test:building/FCU2.timeOfTempe

In [126]:
#save model
model.graph.serialize(destination="sample_building_model.ttl")

<Graph identifier=af1cbf87-1c0e-4867-9431-e67e0dc98d98 (<class 'rdflib.graph.Graph'>)>

# LP: Adding volttron topics using ref schema

In [127]:
from buildingmotif.namespaces import REF
model.graph.bind("ref", REF)

In [128]:
# VLTT = Namespace('https://www.doesntexist.org/volttronsemantics#')
# model.graph.bind("VLTT", VLTT)
# vltt = Library.load(ontology_graph="../../files/volttron_templates.ttl")
# vltt_ref_template = vltt.get_template_by_name(VLTT.VolttronReference)

In [129]:
ref_template = brick.get_template_by_name(REF.TimeseriesReference)

In [130]:
# ref_name = f"volttrontopicname"
# ref_binding_dict = {"name": BLDG[ref_name], "p12":  Literal('this/is/topic')}
# ref_graph = ref_template.evaluate(ref_binding_dict)
# ref_graph.serialize()

In [131]:
ref_template.evaluate(ref_binding_dict).parameters

AttributeError: 'Graph' object has no attribute 'parameters'

In [132]:
for i, row in df.iterrows():
    if row['relation'] == 'hasPoint':
        # ref_binding_dict = {"name": BLDG[row['pointURI'] + '.id'], "p12":  Literal(row['topic'])}
        ref_binding_dict = {"name": BLDG[row['objectURI'] + '.id'], "p27":  Literal(row['topic'])}
        model.add_graph(ref_template.evaluate(ref_binding_dict))
        model.graph.add((BLDG[row['objectURI']], REF['hasExternalReference'], BLDG[row['objectURI'] + '.id']))

In [133]:
model.graph.serialize(destination="building_model.ttl")

<Graph identifier=af1cbf87-1c0e-4867-9431-e67e0dc98d98 (<class 'rdflib.graph.Graph'>)>

In [192]:
query = """
SELECT DISTINCT ?zone ?equip ?Tid ?Tsetid
WHERE {
    ?zone a brick:HVAC_Zone .
    ?equip brick:feeds ?zone ;
    a brick:FCU ;
    brick:hasPoint ?T, ?Tset .

    ?T a brick:Air_Temperature_Sensor ;
      ref:hasExternalReference/ref:hasTimeseriesId ?Tid  .

    ?TSet a brick:Air_Temperature_Setpoint ;
      ref:hasExternalReference/ref:hasTimeseriesId ?TSetid  .
    
}"""


In [193]:
a = model.graph.query(query)

In [194]:
for row in a:
    print(row)

(rdflib.term.URIRef('test:building/Zone1'), rdflib.term.URIRef('test:building/FCU1'), rdflib.term.Literal('sensibo/FCU1/temperature'), None)
(rdflib.term.URIRef('test:building/Zone1'), rdflib.term.URIRef('test:building/FCU2'), rdflib.term.Literal('sensibo/FCU2/temperature'), None)
(rdflib.term.URIRef('test:building/Zone1'), rdflib.term.URIRef('test:building/FCU3'), rdflib.term.Literal('sensibo/FCU3/temperature'), None)
(rdflib.term.URIRef('test:building/Zone1'), rdflib.term.URIRef('test:building/FCU4'), rdflib.term.Literal('sensibo/FCU4/temperature'), None)
(rdflib.term.URIRef('test:building/Zone1'), rdflib.term.URIRef('test:building/FCU5'), rdflib.term.Literal('sensibo/FCU5/temperature'), None)
